In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import datetime
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima_model import ARIMA
import math,random

/Users/liuxichen/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: No module named 'arch'

## Preprocessing

In [ ]:
Testing_Length=150

In [0]:
np.random.seed(101)
test_list=np.zeros(125)
for i in range (0,125):
    if i%2 == 0:
        test_list[i]=-1*np.abs(np.random.normal())
    else:
        test_list[i]=np.abs(np.random.normal())
address=r'C:\Users\liuxichen\Desktop\CS3481 Code\Final_CSV.csv'
InputCSV=pd.read_csv(address,index_col='Dates')
Training_Length=InputCSV.loc['2002/8/1':'2007/9/4'].shape[0]

DJUSGF_simple_return_train=InputCSV['DJUSGF_LAST_PRICE'].pct_change()[1:Training_Length-Testing_Length]
DJUSGF_simple_return_test=InputCSV['DJUSGF_LAST_PRICE'].pct_change()[Training_Length-Testing_Length:Training_Length]

DJUSHC_simple_return_train=InputCSV['DJUSHC_LAST_PRICE'].pct_change()[1:Training_Length-Testing_Length]
DJUSHC_simple_return_test=InputCSV['DJUSHC_LAST_PRICE'].pct_change()[Training_Length-Testing_Length:Training_Length]

SMB_train=InputCSV['SMB'][1:Training_Length-Testing_Length]
SMB_test=InputCSV['SMB'][Training_Length-Testing_Length:Training_Length]

HML_train=InputCSV['HML'][1:Training_Length-Testing_Length]
HML_test=InputCSV['HML'][Training_Length-Testing_Length:Training_Length]

Risk_Free_Rate_train=InputCSV['RF'][1:Training_Length-Testing_Length]
Risk_Free_Rate_test=InputCSV['RF'][Training_Length-Testing_Length:Training_Length]

Market_Excess_train=InputCSV['SPX_LAST_PRICE'].pct_change()[1:Training_Length-Testing_Length]-Risk_Free_Rate_train
Market_Excess_test=InputCSV['SPX_LAST_PRICE'].pct_change()[Training_Length-Testing_Length:Training_Length]-Risk_Free_Rate_test

IndexList=InputCSV.index.tolist()[0:Training_Length+Testing_Length]
sub_Index=IndexList[0:Training_Length]
train_sub_Index=IndexList[0:Training_Length-Testing_Length]
test_sub_Index=IndexList[Training_Length-Testing_Length:Training_Length+1]
test_Index=IndexList[Training_Length:Training_Length+125]

DJUSGF_testing=InputCSV['DJUSGF_LAST_PRICE'].pct_change()[Training_Length:Training_Length+125]
DJUSHC_testing=InputCSV['DJUSHC_LAST_PRICE'].pct_change()[Training_Length:Training_Length+125]

FileNotFoundError: ignored

In [0]:
FamaFrench_Finance=regression.linear_model.OLS(DJUSGF_simple_return_train,sm.add_constant(Market_Excess_train)).fit()
Fama_Mean_Finance=1+FamaFrench_Finance.params['const']+FamaFrench_Finance.params[0]*Market_Excess_train

FamaFrench_Health=regression.linear_model.OLS(DJUSHC_simple_return_train,sm.add_constant(Market_Excess_train)).fit()
Fama_Mean_Health=1+FamaFrench_Health.params['const']+FamaFrench_Health.params[0]*Market_Excess_train

In [0]:
Fama_Mean_Finance_test=1+FamaFrench_Finance.params['const']+FamaFrench_Finance.params[0]*Market_Excess_test
Fama_Mean_Health_test=1+FamaFrench_Health.params['const']+FamaFrench_Health.params[0]*Market_Excess_test

In [0]:
Pred_Fama_Finance=np.zeros(Fama_Mean_Finance.shape[0]+1)
Pred_Fama_Finance[0]=InputCSV['DJUSGF_LAST_PRICE'][0]
Test_Finance=np.zeros(Fama_Mean_Finance_test.shape[0]+1)
Test_Finance[0]=InputCSV['DJUSGF_LAST_PRICE'][Training_Length-Testing_Length]

Pred_Fama_Health=np.zeros(Fama_Mean_Health.shape[0]+1)
Pred_Fama_Health[0]=InputCSV['DJUSHC_LAST_PRICE'][0]
Test_Health=np.zeros(Fama_Mean_Health_test.shape[0]+1)
Test_Health[0]=InputCSV['DJUSHC_LAST_PRICE'][Training_Length-Testing_Length]

In [0]:
## Volatility for Financial Service Index
De_Mean_Finance=DJUSGF_simple_return_train-(Fama_Mean_Finance-1)
De_Mean_Finance_Test=DJUSGF_simple_return_test-(Fama_Mean_Finance_test-1)

model_Finance = arch_model(De_Mean_Finance*100, mean='Zero', vol='GARCH', p=1, q=2)
res_Finance=model_Finance.fit()
epsilon_Finance=(res_Finance.conditional_volatility/100)*np.sign(De_Mean_Finance)

model_Finance_test=arch_model(De_Mean_Finance_Test*100, mean='Zero', vol='GARCH', p=1, q=2)
res_Finance_test=model_Finance_test.fit()
epsilon_Finance_test=(res_Finance_test.conditional_volatility/100)*np.sign(De_Mean_Finance_Test)
##Volatility for Health Care Index
De_Mean_Health=DJUSHC_simple_return_train-(Fama_Mean_Health-1)
De_Mean_Health_Test=DJUSHC_simple_return_test-(Fama_Mean_Health_test-1)

model_Health = arch_model(De_Mean_Health*100, mean='Zero', vol='GARCH', p=1, q=1)
res_Health=model_Health.fit()
epsilon_Health=(res_Health.conditional_volatility/100)*np.sign(De_Mean_Health)

model_Health_test = arch_model(De_Mean_Health_Test*100, mean='Zero', vol='GARCH', p=1, q=1)
res_Health_test=model_Health_test.fit()
epsilon_Health_test=(res_Health_test.conditional_volatility/100)*np.sign(De_Mean_Health_Test)
epsilon_Health_test=epsilon_Health_test/1.5

In [0]:
for i in range(1,Fama_Mean_Finance.shape[0]+1):
    Pred_Fama_Finance[i]=Pred_Fama_Finance[i-1]*(Fama_Mean_Finance[i-1]+epsilon_Finance[i-1])
Pred_Fama_Finance_df=pd.DataFrame(data=Pred_Fama_Finance,index=train_sub_Index,columns=['Finance Index Fama French'])

for i in range(1,Fama_Mean_Finance_test.shape[0]+1):
    Test_Finance[i]=Test_Finance[i-1]*(Fama_Mean_Finance_test[i-1]+epsilon_Finance_test[i-1])
Test_Finance_df=pd.DataFrame(data=Test_Finance,index=test_sub_Index,columns=['Finance Index Fama French Test'])

for i in range(1,Fama_Mean_Health.shape[0]+1):
    Pred_Fama_Health[i]=Pred_Fama_Health[i-1]*(Fama_Mean_Health[i-1]+epsilon_Health[i-1])
Pred_Fama_Health_df=pd.DataFrame(data=Pred_Fama_Health,index=train_sub_Index,columns=['Health Care Index Fama French'])

for i in range(1,Fama_Mean_Health_test.shape[0]+1):
    Test_Health[i]=Test_Health[i-1]*(Fama_Mean_Health_test[i-1]+epsilon_Health_test[i-1])
Test_Health_df=pd.DataFrame(data=Test_Health,index=test_sub_Index,columns=['Health Index Fama French Test'])

In [0]:
%matplotlib inline
plt.rcParams['figure.figsize']=15,10
plt.plot(InputCSV['DJUSGF_LAST_PRICE'][0:Training_Length])
plt.plot(Pred_Fama_Finance_df)
plt.plot(Test_Finance_df)
plt.legend(['Dow Jones U.S. Financial Service Index','F-F Training','F-F Testing'])
plt.xlabel('Time')
plt.ylabel('Dow Jones U.S. Financial Service Index Value')

In [0]:
%matplotlib inline
plt.rcParams['figure.figsize']=15,10
plt.plot(InputCSV['DJUSHC_LAST_PRICE'][0:Training_Length])
plt.plot(Pred_Fama_Health_df)
plt.plot(Test_Health_df)
plt.legend(['Dow Jones U.S. Healthcare Index','F-F Training','F-F Testing'])
plt.xlabel('Time')
plt.ylabel('Dow Jones U.S. Healthcare Index Value')

In [0]:
#De_Mean_Finance.to_csv(r'C:\Users\linzi\Desktop\CS3481 Code\DeMean_CAPM_F.csv')
#De_Mean_Health.to_csv(r'C:\Users\linzi\Desktop\CS3481 Code\DeMean_CAPM_H.csv')
#P1.to_csv(r'C:\Users\linzi\Desktop\CS3481 Code\ARMA_CAMP_F.csv')
#P2.to_csv(r'C:\Users\linzi\Desktop\CS3481 Code\ARMA_CAMP_H.csv')

In [0]:
model = ARIMA((Fama_Mean_Finance+epsilon_Finance)*10000, order=(3,0,2))
model_F = model.fit(disp=0)

model = ARIMA((Fama_Mean_Health+epsilon_Health)*10000, order=(5,0,3))
model_H = model.fit(disp=0)

In [0]:
shock_F=res_Finance.forecast(horizon=125)
epsilon_F_hat=np.sqrt(shock_F.variance.values[-1, :])/100

shock_H=res_Health.forecast(horizon=125)
epsilon_H_hat=np.sqrt(shock_H.variance.values[-1, :])/100

In [0]:
F_fc, F_se, F_conf = model_F.forecast(125, alpha=0.05)  
F_series = (pd.Series(F_fc, index=test_Index)/10000+epsilon_F_hat*test_list)
F_lower_series =(pd.Series(F_conf[:, 0], index=test_Index)/10000-epsilon_F_hat)
F_upper_series = (pd.Series(F_conf[:, 1], index=test_Index)/10000+epsilon_F_hat)

H_fc, H_se, H_conf = model_H.forecast(125, alpha=0.05)  
H_series = (pd.Series(H_fc, index=test_Index)/10000+epsilon_H_hat*test_list)
H_lower_series =(pd.Series(H_conf[:, 0], index=test_Index)/10000-epsilon_H_hat)
H_upper_series = (pd.Series(H_conf[:, 1], index=test_Index)/10000+epsilon_H_hat)

In [0]:
Forecast_Finance=np.zeros(125)
LowerBound_Finance=np.zeros(125)
UpperBound_Finance=np.zeros(125)
Forecast_Finance[0]=InputCSV['DJUSGF_LAST_PRICE'][Training_Length]
LowerBound_Finance[0]=InputCSV['DJUSGF_LAST_PRICE'][Training_Length]
UpperBound_Finance[0]=InputCSV['DJUSGF_LAST_PRICE'][Training_Length]
for i in range(1,125):
    Forecast_Finance[i]=Forecast_Finance[i-1]*(F_series[i-1])
    LowerBound_Finance[i]=Forecast_Finance[i-1]*(F_lower_series[i-1])
    UpperBound_Finance[i]=Forecast_Finance[i-1]*(F_upper_series[i-1])
Dict_F = {'Forecasted Financial Service Index': Forecast_Finance, 'Lower Bound': LowerBound_Finance,'Upper Bound': UpperBound_Finance}
Forecast_Finance_df=pd.DataFrame(data=Dict_F,index=test_Index)

Forecast_Health=np.zeros(125)
LowerBound_Health=np.zeros(125)
UpperBound_Health=np.zeros(125)
Forecast_Health[0]=InputCSV['DJUSHC_LAST_PRICE'][Training_Length]
LowerBound_Health[0]=InputCSV['DJUSHC_LAST_PRICE'][Training_Length]
UpperBound_Health[0]=InputCSV['DJUSHC_LAST_PRICE'][Training_Length]
for i in range(1,125):
    Forecast_Health[i]=Forecast_Health[i-1]*(H_series[i-1])
    LowerBound_Health[i]=Forecast_Health[i-1]*(H_lower_series[i-1])
    UpperBound_Health[i]=Forecast_Health[i-1]*(H_upper_series[i-1])
Dict_H = {'Forecasted Health Care Index': Forecast_Health, 'Lower Bound': LowerBound_Health,'Upper Bound': UpperBound_Health}
Forecast_Health_df=pd.DataFrame(data=Dict_H,index=test_Index)

In [0]:
%matplotlib inline
plt.rcParams['figure.figsize']=15,10
plt.plot(InputCSV['DJUSGF_LAST_PRICE'][Training_Length:Training_Length+125],
         linewidth=5,color='b',label='Dow Jones U.S. Financial Service Index')
plt.plot(Forecast_Finance_df['Forecasted Financial Service Index'],
         linewidth=5,label='Forecasting-Fama French',color='k')
plt.fill_between(test_Index,Forecast_Finance_df['Lower Bound'], Forecast_Finance_df['Upper Bound'],
                 color='k', alpha=.15,label='95% Confidence Interval')
plt.xlabel('Time')
plt.legend()
plt.ylabel('Dow Jones U.S. Financial Service Index Value')

In [0]:
%matplotlib inline
plt.rcParams['figure.figsize']=15,10
plt.ylim(300,420)
plt.plot(InputCSV['DJUSHC_LAST_PRICE'][Training_Length:Training_Length+125],
         linewidth=5,label='Dow Jones U.S. Health Care Index')
plt.plot(Forecast_Health_df['Forecasted Health Care Index'],label='Forecasting-Fama French',linewidth=5)
plt.fill_between(test_Index,Forecast_Health_df['Lower Bound'], Forecast_Health_df['Upper Bound'],
                 color='k', alpha=.15,label='95% Confidence Interval')
plt.xlabel('Time')
plt.legend()
plt.ylabel('Dow Jones U.S. Healthcare Index Value')

In [0]:
## Training Set
Dict_Pred = {'Predicted DJUSGF': Pred_Fama_Finance,
             'DJUSGF': InputCSV['DJUSGF_LAST_PRICE'][0:Training_Length-Testing_Length],
             'Predicted DJUSHC':Pred_Fama_Health,
             'DJUSHC':InputCSV['DJUSHC_LAST_PRICE'][0:Training_Length-Testing_Length]}
Out_Pred_df=pd.DataFrame(data=Dict_Pred,index=train_sub_Index)
Out_Pred_df.to_excel(r'C:\Users\linzi\Desktop\CS3481 Code\CAPM_Train.xlsx')
## Testing Set
Dict_Test = {'Testing DJUSGF': Test_Finance,
             'DJUSGF': InputCSV['DJUSGF_LAST_PRICE'][Training_Length-Testing_Length:Training_Length],
             'Testing DJUSHC':Test_Health,
             'DJUSHC':InputCSV['DJUSHC_LAST_PRICE'][Training_Length-Testing_Length:Training_Length]}
Out_Test_df=pd.DataFrame(data=Dict_Test,index=test_sub_Index)
Out_Test_df.to_excel(r'C:\Users\linzi\Desktop\CS3481 Code\CAPM_Test.xlsx')

In [0]:
## Forecasting
Dict_Fore = {'Forecasted DJUSGF': Forecast_Finance_df['Forecasted Financial Service Index'],
             'DJUSGF Lower Bound': Forecast_Finance_df['Lower Bound'],
             'DJUSGF Upper Bound': Forecast_Finance_df['Upper Bound'],
             'DJUSGF in crisis':InputCSV['DJUSGF_LAST_PRICE'][Training_Length:Training_Length+125],
             'Forecasted DJUSHC': Forecast_Health_df['Forecasted Health Care Index'],
             'DJUSHC Lower Bound': Forecast_Health_df['Lower Bound'],
             'DJUSHC Upper Bound': Forecast_Health_df['Upper Bound'],
             'DJUSHC in crisis':InputCSV['DJUSHC_LAST_PRICE'][Training_Length:Training_Length+125]}
Out_Fore_df=pd.DataFrame(data=Dict_Fore,index=test_Index)
Out_Fore_df.to_excel(r'C:\Users\linzi\Desktop\CS3481 Code\CAPM_Forecasting.xlsx')